# Julia QR
Julia (along with any other high performance library) avoids extraneous computations. The underlying return format for efficient QR code is probably not what you expect. The structure access "." tools compute the pieces you need.  Full documentation is available. 

In [17]:
using LinearAlgebra
(m,n)=(12,5)
A=rand(m,n)
display(@time QROut=qr(A))
display(QROut.Q)
display(Matrix(QROut))

  0.000097 seconds (4 allocations: 1.062 KiB)


LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 12×12 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
5×5 Matrix{Float64}:
 -1.94014  -1.3964   -1.31087   -1.32257   -1.73393
  0.0       1.19548   0.637559   0.652273   0.0449904
  0.0       0.0       1.01946    0.733805   0.0376774
  0.0       0.0       0.0        1.2369     0.198064
  0.0       0.0       0.0        0.0       -0.823469

12×12 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

12×5 Matrix{Float64}:
 0.75553    0.988342    0.677606   0.318105  0.681149
 0.503631   0.0979114   0.177423   0.483035  0.568144
 0.652626   0.659893    0.021569   0.180648  0.884162
 0.873095   0.333092    0.451952   0.345081  0.42291
 0.515644   0.645258    0.706527   0.180684  0.591742
 0.852132   0.356585    0.63535    0.931728  0.561555
 0.0836054  0.00987046  0.80298    0.721608  0.495263
 0.148895   0.265778    0.0192824  0.920664  0.499717
 0.386606   0.469451    0.398514   0.70423   0.272126
 0.67277    0.140243    0.258973   0.129591  0.857502
 0.0701236  0.855856    0.648833   0.879609  0.00351888
 0.351097   0.509411    0.523393   0.448626  0.10031

It is worth noting that the Q factor acts like a square matrix!  It is much faster to use the implicit Q factor than it is to build the Q and then multiply. 

In [37]:
using LinearAlgebra
(m,n)=(1234,25)
A=rand(m,n); vm=rand(m); vn=vm[1:n];
@time QROut=qr(A)
@time Q=QROut.Q
@time uFast=Q*vn
@time uSlow=Matrix(Q)*vn
@time norm(uFast-uSlow)
@time Q*vm;

  0.000585 seconds (5 allocations: 251.266 KiB)
  0.000035 seconds (1 allocation: 32 bytes)
  0.000333 seconds (4 allocations: 10.156 KiB)
  0.001007 seconds (4 allocations: 255.984 KiB)
  0.000016 seconds (2 allocations: 9.828 KiB)
  0.000086 seconds (2 allocations: 10.062 KiB)


The "extra" columns of "Q" span the orthogonal complement of the columns space of A.  This could be useful if you want an efficient way to efficiently expand the column space of A.     

In [56]:
using SparseArrays
@time em = spzeros(m)
@time em[m]=1.0
@time uPerp=Q*em
@time A'*uPerp;

  0.000008 seconds (3 allocations: 160 bytes)
  0.000008 seconds (2 allocations: 160 bytes)
  0.000110 seconds (3 allocations: 19.875 KiB)
  0.000295 seconds (2 allocations: 272 bytes)


Sparse arrays are stored quite differently from dense arrays! Julia gives you a bad cartoon.

In [66]:
(m,n)=(1234,2342)
@time A=zeros(m,n); A[m,n]=2.3; A[1,1]=3.4
@time As=sparse([m,1],[n,1],[2.3,3.4])

  0.005030 seconds (2 allocations: 22.049 MiB)
  0.000032 seconds (18 allocations: 47.328 KiB)


1234×2342 SparseMatrixCSC{Float64, Int64} with 2 stored entries:
⎡⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⎦